# 量化交易系统示例

这个笔记本演示了如何使用量化交易系统进行策略开发和回测。

In [1]:
import sys
from pathlib import Path

# 添加项目根目录到路径
project_root = Path().resolve().parent
sys.path.append(str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 导入自定义模块
from data import DataFetcher, DataManager
from strategies import MovingAverageStrategy, MeanReversionStrategy
from backtesting import BacktestEngine

# 设置matplotlib中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

Loaded environment variables from D:\Work\quant_trading\config\.env


## 1. 数据获取

In [ ]:
# 初始化数据获取器
data_fetcher = DataFetcher()

# 获取苹果公司股票数据
symbol = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-12-31'

print(f"正在获取 {symbol} 的股票数据...")
data = data_fetcher.fetch_stock_data(symbol, start_date, end_date)

print(f"数据形状: {data.shape}")
print(f"日期范围: {data.index[0]} 到 {data.index[-1]}")
data.head()

## 2. 数据可视化

In [ ]:
# 绘制价格图表
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# 价格走势
ax1.plot(data.index, data['close'], label='收盘价', linewidth=1)
ax1.set_title(f'{symbol} 股价走势')
ax1.set_ylabel('价格 ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 成交量
ax2.bar(data.index, data['volume'], alpha=0.6, label='成交量')
ax2.set_title('成交量')
ax2.set_xlabel('日期')
ax2.set_ylabel('成交量')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 移动平均策略

In [ ]:
# 创建移动平均策略
ma_strategy = MovingAverageStrategy(short_window=20, long_window=50)

# 生成交易信号
ma_signals = ma_strategy.generate_signals(data)

print("移动平均策略信号统计:")
print(f"买入信号: {(ma_signals['signal'] == 1).sum()}")
print(f"卖出信号: {(ma_signals['signal'] == -1).sum()}")
print(f"持有信号: {(ma_signals['signal'] == 0).sum()}")

ma_signals.head(60)  # 显示前60天的信号

In [ ]:
# 可视化移动平均策略
fig, ax = plt.subplots(figsize=(14, 8))

# 绘制价格和移动平均线
ax.plot(data.index, data['close'], label='收盘价', linewidth=1, alpha=0.8)
ax.plot(data.index, ma_signals['short_ma'], label='短期MA(20)', linewidth=1)
ax.plot(data.index, ma_signals['long_ma'], label='长期MA(50)', linewidth=1)

# 标记买入卖出信号
buy_signals = data.index[ma_signals['signal'] == 1]
sell_signals = data.index[ma_signals['signal'] == -1]

if len(buy_signals) > 0:
    ax.scatter(buy_signals, data.loc[buy_signals, 'close'], 
              color='green', marker='^', s=100, label='买入信号')

if len(sell_signals) > 0:
    ax.scatter(sell_signals, data.loc[sell_signals, 'close'], 
              color='red', marker='v', s=100, label='卖出信号')

ax.set_title(f'{symbol} - 移动平均交叉策略')
ax.set_xlabel('日期')
ax.set_ylabel('价格 ($)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 回测分析

In [ ]:
# 运行回测
backtest_engine = BacktestEngine(initial_capital=100000)
ma_results = backtest_engine.run_backtest(ma_strategy, data)

print("="*50)
print("移动平均策略回测结果")
print("="*50)
print(f"总收益率: {ma_results['total_return']:.2%}")
print(f"年化收益率: {ma_results['annual_return']:.2%}")
print(f"年化波动率: {ma_results['volatility']:.2%}")
print(f"夏普比率: {ma_results['sharpe_ratio']:.2f}")
print(f"最大回撤: {ma_results['max_drawdown']:.2%}")
print(f"总交易次数: {ma_results['total_trades']}")
print(f"胜率: {ma_results['win_rate']:.2%}")
print(f"盈亏比: {ma_results['profit_factor']:.2f}")
print(f"最终资金: ${ma_results['final_capital']:,.2f}")

## 5. 均值回归策略

In [ ]:
# 创建均值回归策略
mr_strategy = MeanReversionStrategy()

# 生成交易信号
mr_signals = mr_strategy.generate_signals(data)

print("均值回归策略信号统计:")
print(f"买入信号: {(mr_signals['signal'] == 1).sum()}")
print(f"卖出信号: {(mr_signals['signal'] == -1).sum()}")
print(f"持有信号: {(mr_signals['signal'] == 0).sum()}")

# 运行回测
mr_results = backtest_engine.run_backtest(mr_strategy, data)

print("\n" + "="*50)
print("均值回归策略回测结果")
print("="*50)
print(f"总收益率: {mr_results['total_return']:.2%}")
print(f"年化收益率: {mr_results['annual_return']:.2%}")
print(f"年化波动率: {mr_results['volatility']:.2%}")
print(f"夏普比率: {mr_results['sharpe_ratio']:.2f}")
print(f"最大回撤: {mr_results['max_drawdown']:.2%}")
print(f"总交易次数: {mr_results['total_trades']}")
print(f"胜率: {mr_results['win_rate']:.2%}")
print(f"最终资金: ${mr_results['final_capital']:,.2f}")

## 6. 策略对比

In [ ]:
# 创建策略对比表
comparison_data = {
    '指标': ['总收益率', '年化收益率', '年化波动率', '夏普比率', '最大回撤', '胜率', '总交易次数'],
    '移动平均策略': [
        f"{ma_results['total_return']:.2%}",
        f"{ma_results['annual_return']:.2%}",
        f"{ma_results['volatility']:.2%}",
        f"{ma_results['sharpe_ratio']:.2f}",
        f"{ma_results['max_drawdown']:.2%}",
        f"{ma_results['win_rate']:.2%}",
        ma_results['total_trades']
    ],
    '均值回归策略': [
        f"{mr_results['total_return']:.2%}",
        f"{mr_results['annual_return']:.2%}",
        f"{mr_results['volatility']:.2%}",
        f"{mr_results['sharpe_ratio']:.2f}",
        f"{mr_results['max_drawdown']:.2%}",
        f"{mr_results['win_rate']:.2%}",
        mr_results['total_trades']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("策略性能对比:")
print(comparison_df.to_string(index=False))

## 7. 结论

通过这个示例，我们可以看到：

1. **移动平均策略**：适合趋势市场，能够捕捉长期趋势
2. **均值回归策略**：适合震荡市场，能够在价格偏离均值时获利
3. **策略选择**：应根据市场状况选择合适的策略
4. **风险管理**：需要关注最大回撤和夏普比率等风险指标

### 下一步:
- 尝试不同的参数组合
- 组合多个策略
- 添加风险管理模块
- 在更多股票上测试策略的稳定性